In [ ]:
##################### ETL Project #######################

# Extracting,Cleaning and Transforming the CSV file from Azure.

import pandas as pd
from google.colab import userdata,files
import logging
import sys
import os

# --- Setup Logging ---
log_file = 'data_cleaning.log'

# Clean up any existing handlers (important in Colab!)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Create a logger object
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# File handler to log to file
file_handler = logging.FileHandler(log_file, mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
try:
    # Trying to read a CSV file
    csv_url = userdata.get("CSV_URL")

    if not csv_url:
        raise ValueError("CSV_URL secret is not set in Colab.")

    df = pd.read_csv(csv_url)
    logging.info("CSV file loaded successfully!")
    display(df.head())

except ValueError as ve:
    logging.info(f"Value Error: {ve}")

except pd.errors.ParserError as pe:
    logging.info(" Parsing Error: The file could not be parsed as CSV.")
    logging.info(pe)

except FileNotFoundError:
    logging.info("File Not Found: Make sure the URL is correct and accessible.")

except Exception as e:
    logging.info("An unexpected error occurred:")
    logging.info(e)
# --- FLUSHING LOGGING TO FILE ---
for handler in logger.handlers:
    handler.flush()

# --- Show file contents ---
!cat data_cleaning.log

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!


In [ ]:
# Data Cleaning and transformation for CSV file

# Remove duplicates
logger.info("Removing duplicates")
df = df.drop_duplicates()

# Handling missing values
logger.info("Handling missing values")
missing = df.isnull().sum()
print(f"Missing values per column:\n{missing}")

# Filling missing values with default
df=df.fillna({"name":'Unknown',
              "height":'0',
              "weight":'0'})
# Correct data types (ensuring height and weight are in a float format)
logger.info("Ensuring height and weight are in a float format")
df.info()

# --- Show file contents ---
!cat data_cleaning.log

Missing values per column:
name      0
height    0
weight    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    5 non-null      object 
 1   height  5 non-null      float64
 2   weight  5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 252.0+ bytes
2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format


In [ ]:
# --- Unit Conversion ---
# Inches to meters: 1 inch = 0.0254 meters
# Pounds to kilograms: 1 pound = 0.453592 kilograms

if 'height' in df.columns:
    df['height_m'] = df['height'] * 0.0254
    logger.info("Converted height from inches to meters.")

if 'weight' in df.columns:
    df['weight_kg'] = df['weight'] * 0.453592
    logger.info("Converted weight from pounds to kilograms.")

# --- Standardizing Column Names ---
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
logger.info("Standardized column names to lowercase with underscores.")

# --- Drop 'height' and 'weight' columns if they exist ---
columns_to_drop = ['height', 'weight']
existing_cols = [col for col in columns_to_drop if col in df.columns]
if existing_cols:
    df.drop(columns=existing_cols, inplace=True)
    logger.info(f"Dropped columns: {existing_cols}")
else:
    logger.info("No 'height' or 'weight' columns found to drop")

# --- Round height and weight to 2 decimal places ---
if 'height_m' in df.columns:
    df['height_m'] = df['height_m'].round(2)
    logger.info("Rounded 'height' to 2 decimal places.")

if 'weight_kg' in df.columns:
    df['weight_kg'] = df['weight_kg'].round(2)
    logger.info("Rounded 'weight' to 2 decimal places.")


# --- Preview of cleaned and transformed data ---
display(df.head())

# --- Show file contents ---
!cat data_cleaning.log

,name,height_m,weight_kg
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.


In [ ]:
# Save cleaned DataFrame to local CSV and then upload to Azure
df.to_csv("transformed_data.csv")

import requests

# Replace with your SAS URL
sas_url = userdata.get("SAS_CSV_URL")

# Open the file in binary read mode
with open("transformed_data.csv", "rb") as data:
    headers = {
        "x-ms-blob-type": "BlockBlob"
    }
    response = requests.put(sas_url, headers=headers, data=data)

# Check the response status
if response.status_code == 201:
    logger.info("Transformed CSV file uploaded to Azure successfully.")
else:
    logger.info(f"Upload failed. Status code: {response.status_code}")
    logger.info(response.text)

# --- Show file contents ---
!cat data_cleaning.log

2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.
2025-04-19 19:32:33,220 - INFO - Transformed CSV file uploaded to Azure successfully.


In [ ]:
# Reading the cleaned and tranformed csv file from Azure.

transformed_csv_url = userdata.get("T_CSV")
df_cleaned_csv = pd.read_csv(transformed_csv_url)
logging.info("Retrive the transformed CSV file from Azure")
display(df_cleaned_csv.head())

# --- Show file contents ---
!cat data_cleaning.log

,Unnamed: 0,name,height_m,weight_kg
0,0,alex,1.67,51.25
1,1,ajay,1.82,61.91
2,2,alice,1.76,69.41
3,3,ravi,1.73,64.56
4,4,joe,1.72,65.45


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.
2025-04-19 19:32:33,220 - INFO - Transformed CSV file uploaded to Azure successfully.
2025-04-19 19:32:54,414 - INFO - Retrive the transformed CSV file from Azure


In [ ]:
#################  JSON  ##################
#Extracting,Cleaning and Transforming the JSON file from Azure.

import pandas as pd
from google.colab import userdata

try:
    # Trying to read JSON file
    json_url = userdata.get("JSON_URL")

    if not json_url:
        raise ValueError("JSON_URL secret is not set in Colab.")

    # For line-delimited JSON
    df_json = pd.read_json(json_url, lines=True)
    logging.info("JSON file loaded successfully!")
    display(df_json.head())

except ValueError as ve:
    logger.info(f"Value Error: {ve}")

except pd.errors.ParserError as pe:
    logger.info(" Parsing Error: The file could not be parsed as JSON.")
    logger.info(pe)

except FileNotFoundError:
    logger.info("File Not Found: Make sure the URL is correct and accessible.")

except Exception as e:
    logger.info("An unexpected error occurred:")
    logger.info(e)


,name,height,weight
0,jack,68.70,123.30
1,tom,69.80,141.49
2,tracy,70.01,136.46
3,john,67.90,112.37


In [ ]:
# Data Cleaning and transformation for JSON file

# Remove duplicates
logger.info("Removing duplicates")
df_json = df_json.drop_duplicates()

# Handling missing values
logger.info("Handling missing values")
missing = df_json.isnull().sum()
print(f"Missing values per column:\n{missing}")

# Filling missing values with default
df_json=df_json.fillna({"name":'Unknown',
              "height":'0',
              "weight":'0'})
# Correct data types (ensuring height and weight are in a float format)
logger.info("Ensuring height and weight are in a float format")
df_json.info()


Missing values per column:
name      0
height    0
weight    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    4 non-null      object 
 1   height  4 non-null      float64
 2   weight  4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 228.0+ bytes


In [ ]:
# --- Unit Conversion ---
# Inches to meters: 1 inch = 0.0254 meters
# Pounds to kilograms: 1 pound = 0.453592 kilograms

if 'height' in df_json.columns:
    df_json['height_m'] = df_json['height'] * 0.0254
    logger.info("Converted height from inches to meters.")

if 'weight' in df_json.columns:
    df_json['weight_kg'] = df_json['weight'] * 0.453592
    logger.info("Converted weight from pounds to kilograms.")

# --- Standardizing Column Names ---
df_json.columns = df_json.columns.str.strip().str.lower().str.replace(' ', '_')
logger.info("Standardized column names to lowercase with underscores.")

# --- Drop 'height' and 'weight' columns if they exist ---
columns_to_drop = ['height', 'weight']
existing_cols = [col for col in columns_to_drop if col in df_json.columns]
if existing_cols:
    df_json.drop(columns=existing_cols, inplace=True)
    logger.info(f"Dropped columns: {existing_cols}")
else:
    logger.info("No 'height' or 'weight' columns found to drop")

# --- Round height and weight to 2 decimal places ---
if 'height_m' in df_json.columns:
    df_json['height_m'] = df_json['height_m'].round(2)
    logger.info("Rounded 'height' to 2 decimal places.")

if 'weight_kg' in df_json.columns:
    df_json['weight_kg'] = df_json['weight_kg'].round(2)
    logger.info("Rounded 'weight' to 2 decimal places.")


# --- Preview of cleaned and transformed data ---
display(df_json.head())


,name,height_m,weight_kg
0,jack,1.74,55.93
1,tom,1.77,64.18
2,tracy,1.78,61.90
3,john,1.72,50.97


In [ ]:
# Save cleaned DataFrame to local json and then uploaded to Azure.

df_json.to_json("transformed_data.json")

import requests

# Replace with your SAS URL
sas_url = userdata.get("SAS_JSON_URL")

# Open the file in binary read mode
with open("transformed_data.json", "rb") as data:
    headers = {
        "x-ms-blob-type": "BlockBlob"
    }
    response = requests.put(sas_url, headers=headers, data=data)

# Check the response status
if response.status_code == 201:
    logger.info("Transformed JSON file uploaded to Azure successfully.")
else:
    logger.info(f"Upload failed. Status code: {response.status_code}")
    logger.info(response.text)


In [ ]:
# Reading the cleaned and tranformed JSON file from Azure.

transformed_json_url = userdata.get("T_JSON")
df_cleaned_json = pd.read_json(transformed_json_url)
logging.info("Retrive the transformed JSON file from Azure")
display(df_cleaned_json.head())

# --- Show file contents ---
!cat data_cleaning.log

,name,height_m,weight_kg
0,jack,1.74,55.93
1,tom,1.77,64.18
2,tracy,1.78,61.90
3,john,1.72,50.97


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.
2025-04-19 19:32:33,220 - INFO - Transformed CSV file uploaded to Azure successfully.
2025-04-19 19:32:54,414 - INFO - Retrive the transformed CSV file from Azure
2025-04-19 19:34:14,746 - INFO - JSON file loaded successfully!
2025-04-19 19:34:27,575 - INFO - Removing duplicates
2025-04-19 19

In [ ]:
#################  XML  ########################

# Extracting,Cleaning and Transforming the XML file from Azure.

import pandas as pd
from google.colab import userdata

try:
    # Trying to read source xml file
    xml_url = userdata.get("XML_URL")

    if not xml_url:
        raise ValueError("XML_URL secret is not set in Colab.")

    # Try reading the XML file directly
    df_xml = pd.read_xml(xml_url)
    logger.info(" XML file loaded successfully!")
    display(df_xml.head())

except ValueError as ve:
    logger.info(f"Value Error: {ve}")

except FileNotFoundError:
    logger.info("File Not Found: Check if the XML URL is correct and accessible.")

except pd.errors.ParserError as pe:
    logger.info("Parsing Error: The file could not be parsed as XML.")
    logger.info(pe)

except Exception as e:
    logger.info("An unexpected error occurred:")
    logger.info(e)


,name,height,weight
0,simon,67.90,112.37
1,jacob,66.78,120.67
2,cindy,66.49,127.45
3,ivan,67.62,114.14


In [ ]:
# Data Cleaning and transformation for XML file

# Remove duplicates
logger.info("Removing duplicates")
df_xml = df_xml.drop_duplicates()

# Handling missing values
logger.info("Handling missing values")
missing = df_xml.isnull().sum()
print(f"Missing values per column:\n{missing}")

# Filling missing values with default
df_xml=df_xml.fillna({"name":'Unknown',
              "height":'0',
              "weight":'0'})
# Correct data types (ensuring height and weight are in a float format)
logger.info("Ensuring height and weight are in a float format")
df_xml.info()
display(df_xml.head())

Missing values per column:
name      0
height    0
weight    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    4 non-null      object 
 1   height  4 non-null      float64
 2   weight  4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 228.0+ bytes


,name,height,weight
0,simon,67.90,112.37
1,jacob,66.78,120.67
2,cindy,66.49,127.45
3,ivan,67.62,114.14


In [ ]:
# --- Unit Conversion ---
# Inches to meters: 1 inch = 0.0254 meters
# Pounds to kilograms: 1 pound = 0.453592 kilograms

if 'height' in df_xml.columns:
    df_xml['height_m'] = df_xml['height'] * 0.0254
    logger.info("Converted height from inches to meters.")

if 'weight' in df_xml.columns:
    df_xml['weight_kg'] = df_xml['weight'] * 0.453592
    logger.info("Converted weight from pounds to kilograms.")

# --- Standardizing Column Names ---
df_xml.columns = df_xml.columns.str.strip().str.lower().str.replace(' ', '_')
logger.info("Standardized column names to lowercase with underscores.")

# --- Drop 'height' and 'weight' columns if they exist ---
columns_to_drop = ['height', 'weight']
existing_cols = [col for col in columns_to_drop if col in df_xml.columns]
if existing_cols:
    df_xml.drop(columns=existing_cols, inplace=True)
    logger.info(f"Dropped columns: {existing_cols}")
else:
    logger.info("No 'height' or 'weight' columns found to drop")

# --- Round height and weight to 2 decimal places ---
if 'height_m' in df_xml.columns:
    df_xml['height_m'] = df_xml['height_m'].round(2)
    logger.info("Rounded 'height' to 2 decimal places.")

if 'weight_kg' in df_xml.columns:
    df_xml['weight_kg'] = df_xml['weight_kg'].round(2)
    logger.info("Rounded 'weight' to 2 decimal places.")


# --- Preview of cleaned and transformed data ---
display(df_xml.head())


,name,height_m,weight_kg
0,simon,1.72,50.97
1,jacob,1.70,54.73
2,cindy,1.69,57.81
3,ivan,1.72,51.77


In [ ]:
# Save cleaned DataFrame to local XML and then upload to Azure
df_xml.to_xml("transformed_data.xml")

import requests

# Replace with your SAS URL
sas_url = userdata.get("SAS_XML_URL")

# Open the file in binary read mode
with open("transformed_data.xml", "rb") as data:
    headers = {
        "x-ms-blob-type": "BlockBlob"
    }
    response = requests.put(sas_url, headers=headers, data=data)

# Check the response status
if response.status_code == 201:
    logger.info("Transformed XML file uploaded to Azure successfully.")
else:
    logger.info(f"Upload failed. Status code: {response.status_code}")
    logger.info(response.text)


In [ ]:
# Reading the cleaned and tranformed xML file from Azure.
transformed_xml_url = userdata.get("T_XML")
df_cleaned_xml = pd.read_xml(transformed_xml_url)
logging.info("Retrive the transformed XML file from Azure")
display(df_cleaned_xml.head())

# --- Show file contents ---
!cat data_cleaning.log

,index,name,height_m,weight_kg
0,0,simon,1.72,50.97
1,1,jacob,1.70,54.73
2,2,cindy,1.69,57.81
3,3,ivan,1.72,51.77


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.
2025-04-19 19:32:33,220 - INFO - Transformed CSV file uploaded to Azure successfully.
2025-04-19 19:32:54,414 - INFO - Retrive the transformed CSV file from Azure
2025-04-19 19:34:14,746 - INFO - JSON file loaded successfully!
2025-04-19 19:34:27,575 - INFO - Removing duplicates
2025-04-19 19

In [ ]:
############# Loading the data to Azure MySQL database ##################

# Installing pymysql package
! pip install pymysql

import pandas as pd
from sqlalchemy import create_engine

# Getting the connection details from colabs secret keys
conn_url=userdata.get("DBCONN_URL")

engine = create_engine(f"{conn_url}",
                       connect_args={
        "ssl": {
            "ssl_verify_cert": False
        }
    })
# Loading the CSV file
df.to_sql(name='etlcsv', con=engine, if_exists='replace', index=False)
logger.info("CSV file loaded to Azure MySQL database successfully!")

# Loading the JSON file
df_json.to_sql(name='etljson', con=engine, if_exists='replace', index=False)
logger.info("JSON file loaded to Azure MySQL database successfully!")

# Loading the XML file
df_xml.to_sql(name='etlxml', con=engine, if_exists='replace', index=False)
logger.info("XML file loaded to Azure MySQL database successfully!")

# --- Show file contents ---
!cat data_cleaning.log


2025-04-19 19:29:51,238 - INFO - CSV file loaded successfully!
2025-04-19 19:30:40,162 - INFO - Removing duplicates
2025-04-19 19:30:40,165 - INFO - Handling missing values
2025-04-19 19:30:40,168 - INFO - Ensuring height and weight are in a float format
2025-04-19 19:30:52,525 - INFO - Converted height from inches to meters.
2025-04-19 19:30:52,526 - INFO - Converted weight from pounds to kilograms.
2025-04-19 19:30:52,527 - INFO - Standardized column names to lowercase with underscores.
2025-04-19 19:30:52,529 - INFO - Dropped columns: ['height', 'weight']
2025-04-19 19:30:52,530 - INFO - Rounded 'height' to 2 decimal places.
2025-04-19 19:30:52,530 - INFO - Rounded 'weight' to 2 decimal places.
2025-04-19 19:32:33,220 - INFO - Transformed CSV file uploaded to Azure successfully.
2025-04-19 19:32:54,414 - INFO - Retrive the transformed CSV file from Azure
2025-04-19 19:34:14,746 - INFO - JSON file loaded successfully!
2025-04-19 19:34:27,575 - INFO - Removing duplicates
2025-04-19 19

In [ ]:
# Retriving the CSV file from Azure databse
query = "SELECT * FROM etlcsv;"
csv = pd.read_sql(query, con=engine)
logger.info("CSV file retrieved from Azure MySQL database successfully!")

# Show the result
print(csv)

    name  height_m  weight_kg
0   alex      1.67      51.25
1   ajay      1.82      61.91
2  alice      1.76      69.41
3   ravi      1.73      64.56
4    joe      1.72      65.45


In [ ]:
# Retriving the JSON file from Azure databse
query = "SELECT * FROM etljson;"
json = pd.read_sql(query, con=engine)
logger.info("JSON file retrieved from Azure MySQL database successfully!")

# Show the result
print(json)

    name  height_m  weight_kg
0   jack      1.74      55.93
1    tom      1.77      64.18
2  tracy      1.78      61.90
3   john      1.72      50.97


In [ ]:
# Retriving the XML file from Azure databse
query = "SELECT * FROM etlxml;"
xml = pd.read_sql(query, con=engine)
logger.info("XML file retrieved from Azure MySQL database successfully!")

# Show the result
print(xml)

    name  height_m  weight_kg
0  simon      1.72      50.97
1  jacob      1.70      54.73
2  cindy      1.69      57.81
3   ivan      1.72      51.77


In [ ]:
from google.colab import files
files.download('data_cleaning.log')
logger.info("Log file downloaded successfully!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>